In [6]:
# Importación de las librerías de los widgets, numpy para el manejo de vectores y operaciones con arreglos, pandas para la 
# extracción de tablas y características, io para funciones del sistema y display para mostrar los widgets en nuestro 
# notebook
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import random

from io import StringIO
from IPython.display import display
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import Perceptron
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from statistics import mode
from sklearn.preprocessing import StandardScaler


In [19]:
# Delimitador de los valores en el archivo
delim = ","

#Número de filas que se excluyen del set de datos (Observaciones, atributos y clases)
rows = 3

#Widget para ingresar un valor entero
w = widgets.IntSlider(
    value=2,
    min=2,
    max=12,
    step=1,
    description='#',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
#Ingresar en número de pliegues
Folds = widgets.IntSlider(
    value=2,
    min=2,
    max=10,
    step=1,
    description='#',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
#Botón de comienzo del ensamble
Start = widgets.Button(
        description='Comenzar',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Comenzar ensamble',
        icon='check' # (FontAwesome names without the `fa-` prefix)
        )
# #Botón para calcular PCA sobre los datos
# calc = widgets.Button(
#         description='Calcular PCA',
#         disabled=False,
#         button_style='', # 'success', 'info', 'warning', 'danger' or ''
#         tooltip='Calcular PCA sobre los datos para NAive BAyes y Perceoptrón',
#         icon='check' # (FontAwesome names without the `fa-` prefix)
#         )

#Widget para el botón que inicia la secuencia del PCA
b = widgets.Button(
    description='Cargar datos',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Cargar datos',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

#Widget de salida
output = widgets.Output()


#Widget para subir el archivo
up= widgets.FileUpload(
    accept='',  # Archivos o extenciones válidas. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # Cambiar a verdadero para aceptar más de un archivo pero no está soportado por la secuencia de lectura
)

# Convertidor del archivo
# Cuando se usa el widget para subir archivos, el formato en el que arroja el documento no puede usarse directamente en la 
# función read_csv de pandas ya que la estructura es un JSON.

# Se tiene que extraer el contenido de una serie de etiquetas, siendo la última (content) la
# importante. Esta función sólo extrae de la etiqueta content el archivo leido para luego pasarlo a una cadena o buffer
# con formato utf-8 que la función read_csv pueda leer.

def content_parser():
    # Si el contenido de up no tiene nada, entonces se manda el mensaje de error
    
    if up.value == {}:
        # with output se encarga de mostrar valores en el widget de salida (output)
        with output:
            print('No CSV loaded')    
    else:
        # Inicialización de las variables typ y content
        typ, content = "", ""
        
        # Extracción del contenido del widget up
        up_value = up.value
        
        # Recorre internamente la estructura del JSON que se obtuvo del widget up
        for i in up_value.keys():
            
            # Del valor value de nuestra variable up se extrae el componente type de metadata para verificar que es un
            # archivo soportado
            
            typ = up_value[i]["metadata"]["type"]
            
            # Verificación del tipo de archivo
            
            if typ == "application/vnd.ms-excel" or typ == "text/plain":
                # Extracción del contenido de la etiqueta content
                content = up_value[i]["content"]
                content_str = str(content, 'utf-8')
                
                if content_str != "":
                    str_io = StringIO(content_str) 
                    return str_io

def on_b_clicked(b):
    # Llamado a la función que permite parsear el contenido
    content = content_parser()
    
    #Si no se regresa un valor parseado entonces no se ejecuta nada de código y sólo se muestra el mensaje de error
    if content is not None:
        # Lectura del csv desde el buffer de memoria content, quitando los indices de las columnas, estableciendo el
        # delimitador del contenido y quitando el header que pueda 
        data = pd.read_csv(content, index_col=False, sep=delim, header=None)
        
        # Extracción de las dimensiones de nuestro archivo, para poder determinar el final de las características y de las
        # observaciones
        xDim, yDim = data.shape
        
        # Extracción del número de observaciones, atributos y clases de nuestros datos
        noObservations = data[0][0]    
        noAttributes = data[0][1]
        noClasses = data[0][2]
        
        # Impresión de los valores obtenidos
        print("Información de los datos cargados: ")
        print("# de Observaciones {}".format(noObservations))
        print("# de Atributos {}".format(noAttributes))
        print("# de Clases {}".format(noClasses))
        
        w.max = noAttributes
        # Extracción de las primeras n columnas que corresponden a las características de los datos
        # Se elimina también la columna de los índices para reiniciar el contador y no empezar desde 3
        # Se debe tomar en cuenta que la dimensión en Y es mayor que la dimensión total, por ejemplo, si el dataset
        # tiene de 0 a 15 características, entonces la dimensión de Y será de 16, por eso se le resta uno a la dimensión Y
        global dataSet
        dataSet = data.iloc[rows:,:yDim].reset_index(drop=True)
        
        # Extracción de la columna que contiene a las clases de nuestro data set
        # Se sigue el mismo proceso, pero ahora se mantiene sólo la última columna. Se deja entre [] para que el resultado
        # se quede en las mismas condiciones que el anterior
        global classes
        classes = data.iloc[rows:,yDim - 1 ].reset_index(drop=True)
        # Impresión del data set (primeros 5 valores) y de las clases
        print(dataSet.head())
        print("\n")
        print(classes.head())
        print("\n")
        display(Start)
                
#Función para ejecutarse cuando el botón "startPCA" se active
def on_Start_clicked(Start):
    
        NoComp = w.value
        #dataSet.head()
        pca = PCA(n_components=NoComp)
        
        print("Comenzando PCA con los componentes elegidos\n")
        
        global X_std
        X_std = StandardScaler().fit_transform(dataSet)

        pca.fit(X_std)
        global DatosPCA
        DatosPCA = pca.transform(X_std)
    
        print("Radio de varianza")
        print(pca.explained_variance_ratio_)
        print("\n")
        print("Valores singulares")
        print(pca.singular_values_)
        print("\n")
        print("Matriz de proyección")
        print((pca.components_).T)
        print("\n")
        folds = Folds.value
        # Naive Bayes
        X = DatosPCA
        y = classes.values
    
        #Arreglos de predicciones del ensamble para un elemento
        y_pred_Ensamble = []
    
        #Arreglos de efectividad parciales de los clasificadores
        ParcialesBayes = []
        ParcialesPercep = []
        ParcialesArbol = []
        ParcialesEnsamble = []
    
        #Desea mezclar las muestras?
        kf = StratifiedKFold( n_splits = folds , shuffle = True , random_state = None )
    
    #     for train_index, test_index in kf.split(X, y):
    #         #print("TRAIN:", train_index, "TEST:", test_index)
    #         X_train, X_test = X[train_index], X[test_index]    
    #         y_train, y_test = y[train_index], y[test_index]
    
        #TODO: SEPARAR EL ÁRBOL DE DECISIÓN PARA NO APLICARLE PCA
    #    for i,split in enumerate kf.split(X, y):
        for train_index, test_index in kf.split(X, y):
            #print("TRAIN:", train_index, "TEST:", test_index)
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
        
            #print( "    Comienzo del entrenamiento, vuelta {}".format(i))
            #Bayes
            bayes_ingenuo = GaussianNB()
            # predicción 
            y_pred = bayes_ingenuo.fit(X_train, y_train).predict(X_test)
            # Matriz de confusión
            cnf_matrix = confusion_matrix(y_test, y_pred)
            ResBayes = 1 - (y_test != y_pred).sum()/y_test.shape[0]
            ParcialesBayes.append(ResBayes)
            print("Aplicando Naive Bayes sobre datos luego del PCA")
            print("Cantidad de errores de clasificación sobre un total de {0} casos: {1}"
                    .format(y_test.shape[0],(y_test != y_pred).sum()))
            print("Efectividad del algoritmo: {0: .2f}"
                    .format(ResBayes))
            # Graficando la matriz de confusión
            sns.heatmap(cnf_matrix.T, square=True, annot=True, fmt='d', cbar=False)
            plt.xlabel('Clase verdadera')
            plt.ylabel('Clase predecida')
            plt.title('Matriz de Confusión')
            plt.show()

            #Perceptron
            ppn = Perceptron( eta0=0.1, random_state=0)
            ppn.fit(X_train, y_train)
            y_predA = ppn.predict(X_test)
        
            cnf_matrix2 = confusion_matrix(y_test, y_predA)
            ResPercep = 1 - (y_test != y_predA).sum()/y_test.shape[0]
            ParcialesPercep.append(ResPercep)
            print("Aplicando Perceptrón sobre datos luego del PCA")
            print("Número de iteraciones")
            print(ppn.n_iter_)
            print("Cantidad de errores de clasificación sobre un total de {0} casos: {1}"
                    .format(y_test.shape[0],(y_test != y_predA).sum()))
            print("Efectividad del algoritmo: {0: .2f}"
                    .format(ResPercep))
            # Graficando la matriz de confusión
            sns.heatmap(cnf_matrix2.T, square=True, annot=True, fmt='d', cbar=False)
            plt.xlabel('Clase verdadera')
            plt.ylabel('Clase predecida')
            plt.title('Matriz de Confusión')
            plt.show()
    
    
            #Arbol de decisión
            clf = DecisionTreeClassifier()
            clf = clf.fit(X_train ,y_train)
            y_predP = clf.predict(X_test)

            cnf_matrix3 = confusion_matrix(y_test, y_predP)
            ResArbol = 1 - (y_test != y_predP).sum()/y_test.shape[0]
            ParcialesArbol.append(ResArbol)
            print("Aplicando Árbol de decisión sobre datos luego del PCA")
            print("Cantidad de errores de clasificación sobre un total de {0} casos: {1}"
                    .format(y_test.shape[0],(y_test != y_predP).sum()))
            print("Efectividad del algoritmo: {0: .2f}"
                    .format(ResArbol))
            # Graficando la matriz de confusión
            sns.heatmap(cnf_matrix3.T, square=True, annot=True, fmt='d', cbar=False)
            plt.xlabel('Clase verdadera')
            plt.ylabel('Clase predecida')
            plt.title('Matriz de Confusión')
            plt.show()
    
            #Ensamble sobre resultados parciales
    
            for i in range(len(y_pred)):
                y1 = y_pred[i]
                y2 = y_predP[i]
                y3 = y_predA[i]
                x = [y1, y2, y3]
                if y1 != y2 and y1 != y3 and y2 != y3 :
                    y_pred_Ensamble.append(random.choice(x))
                else :
                    y_pred_Ensamble.append(mode(x))

            cnf_matrixEnsamble = confusion_matrix(y_test, y_pred_Ensamble)
            ResEnsamble = 1 - (y_test != y_pred_Ensamble).sum()/y_test.shape[0]
            ParcialesEnsamble.append(ResEnsamble)
            print("Aplicando el ensamble, resultados parciales")
            print("Cantidad de errores de clasificación sobre un total de {0} casos: {1}"
                    .format(y_test.shape[0],(y_test != y_pred_Ensamble).sum()))
            print("Efectividad del algoritmo: {0: .2f}"
                    .format(ResEnsamble))
            # Graficando la matriz de confusión
            sns.heatmap(cnf_matrixEnsamble.T, square=True, annot=True, fmt='d', cbar=False)
            plt.xlabel('Clase verdadera')
            plt.ylabel('Clase predecida')
            plt.title('Matriz de Confusión')
            plt.show()
            y_pred_Ensamble = []

        print("Resultados de efectividad promedio:")
        avgBay = sum(ParcialesBayes)/len(ParcialesBayes)
        avgPer = sum(ParcialesPercep)/len(ParcialesPercep)
        avgArb = sum(ParcialesArbol)/len(ParcialesArbol)
        avgEns = sum(ParcialesEnsamble)/len(ParcialesEnsamble)

        print("Naive Bayes: ")
        print(avgBay)

        print("Perceptrón: ")
        print(avgPer)

        print("Árbol de decisión: ")
        print(avgArb)

        print("Ensamble aplicado: ")
        print(avgEns)
           
    
    # Asignación de la función on_button_clicked al evento on_click del botón b 
Start.on_click(on_Start_clicked)
b.on_click(on_b_clicked)

In [22]:
print("Cargue el archivo .txt o .csv que contiene los datos a analizar y presione el botón cuando esté listo")
display(up, b)
print("\nIngrese el número de componentes para el proceso de PCA")
display(w)
print("\nElija el número de pliegues para la validación cruzada")
display(Folds, output)
print("\n\nSi los datos son correctos, presione el botón para comenzar \n\n")

Cargue el archivo .txt o .csv que contiene los datos a analizar y presione el botón cuando esté listo


FileUpload(value={'Slice409.txt': {'metadata': {'name': 'Slice409.txt', 'type': 'text/plain', 'size': 619551, …

Button(description='Cargar datos', icon='check', style=ButtonStyle(), tooltip='Cargar datos')


Ingrese el número de componentes para el proceso de PCA


IntSlider(value=49, continuous_update=False, description='#', max=385, min=2)


Elija el número de pliegues para la validación cruzada


IntSlider(value=3, continuous_update=False, description='#', max=10, min=2)

Output()



Si los datos son correctos, presione el botón para comenzar 




In [9]:
# kf = StratifiedKFold( n_splits = 2 , shuffle = True , random_state = None )
# for i,split in enumerate (kf.split(X, y)):
#     print (i)
#     print("Dato X = 0")
#     print(split[0])
#     print("Dato X = 1")
#     print(split[1])
    
# print("Parte dos :D")
# for train_index, test_index in kf.split(X, y):
#     print("TRAIN:", train_index, "\nTEST:", test_index)
#     X_train, X_test = X[train_index], X[test_index]
#     y_train, y_test = y[train_index], y[test_index]   



    
    #Módulo para ver los datos del PCA antes y después de normalizar con 2 componentes
#    if(NoComp == 2):
#         Y = pca.transform(X_std)
#         y = classes.iloc[:,0].values
    
#         with plt.style.context('seaborn-whitegrid'):
#             plt.figure(figsize=(6, 4))
#             for lab, col in zip((0, 1, 2, 3, 4, 5, 6),
#                         ('magenta', 'cyan', 'limegreen', 'blue', 'red', 'pink', 'black')):
#                 plt.scatter(Y[y==lab, 0],
#                         -Y[y==lab, 1],
#                         label=lab,
#                         c=col)
#         plt.title("Datos normalizados")
#         plt.xlabel('Componente Principal 1')
#         plt.ylabel('Componente Principal 2')
#         plt.legend(loc='best')
#         plt.tight_layout()
#         plt.show()
    
#         pca.fit(dataSet)


#         print("Radio de varianza")
#         print(pca.explained_variance_)
#         print("Valores singulares")
#         print(pca.singular_values_)

#         Y = pca.transform(dataSet)
#         y = classes.iloc[:,0].values
    
#         #(pca.components_).T

#         with plt.style.context('seaborn-whitegrid'):
#             plt.figure(figsize=(6, 4))
#             for lab, col in zip((0, 1, 2, 3, 4, 5),
#                         ('magenta', 'cyan', 'limegreen', 'blue', 'red', 'pink')):
#                 plt.scatter(Y[y==lab, 0],
#                             -Y[y==lab, 1],
#                             label=lab,
#                             c=col)
            
#         plt.title("Datos no normalizados")
#         plt.xlabel('Componente Principal 1')
#         plt.ylabel('Componente Principal 2')
#         plt.legend(loc='best')
#         plt.tight_layout()
#         plt.show()

In [10]:
# datas = pd.read_csv("PruebaPy.csv", skiprows=3, header=None)
# xDim, yDim = datas.shape
# XD = datas.iloc[:,0:yDim-2].values
# YD = datas.iloc[:,yDim-1].values

# print(XD)
# print(YD)


# from sklearn.preprocessing import StandardScaler
# X_stdP = StandardScaler().fit_transform(XD)
# pca = PCA(n_components=8)
# pca.fit(X_stdP)

# XDP = pca.transform(X_stdP)
# print(XDP)


# #NewCode
# import seaborn as sns
# from sklearn.model_selection import train_test_split
# from sklearn.naive_bayes import GaussianNB
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.linear_model import Perceptron
# from sklearn.metrics import confusion_matrix
# #NewCode 2
# from sklearn.model_selection import KFold
# from sklearn.model_selection import StratifiedKFold


# # Naive Bayes
# X = XDP
# y = YD

# y_pred_Ensamble = []
# y_pred_Ensamble2 = []
# y_pred_Ensamble3 = []
# ParcialesBayes = []
# ParcialesPercep = []
# ParcialesArbol = []
# ParcialesEnsamble = []

# from statistics import mode

# kf = StratifiedKFold( n_splits = 5 , shuffle = True , random_state = None )
# for train_index, test_index in kf.split(X, y):
#     #print("TRAIN:", train_index, "TEST:", test_index)
#     X_train, X_test = X[train_index], X[test_index]
#     y_train, y_test = y[train_index], y[test_index]
#     print( "    Comienzo del entrenamiento   ")
#     #Bayes
#     bayes_ingenuo = GaussianNB()
#     # predicción 
#     y_pred = bayes_ingenuo.fit(X_train, y_train).predict(X_test)
#     # Matriz de confusión
#     cnf_matrix = confusion_matrix(y_test, y_pred)
#     ResBayes = 1 - (y_test != y_pred).sum()/y_test.shape[0]
#     ParcialesBayes.append(ResBayes)
#     print("Aplicando Naive Bayes sobre datos luego del PCA")
#     print("Cantidad de errores de clasificación sobre un total de {0} casos: {1}"
#             .format(y_test.shape[0],(y_test != y_pred).sum()))
#     print("Efectividad del algoritmo: {0: .2f}"
#             .format(ResBayes))
#     # Graficando la matriz de confusión
#     sns.heatmap(cnf_matrix.T, square=True, annot=True, fmt='d', cbar=False)
#     plt.xlabel('Clase verdadera')
#     plt.ylabel('Clase predecida')
#     plt.title('Matriz de Confusión')
#     plt.show()
    
# #     sns.stripplot( x = y_test, y = y_pred, jitter = 0.9)
# #     plt.show()
    
#     #Perceptron
#     ppn = Perceptron( eta0=0.1, random_state=0)
#     ppn.fit(X_train, y_train)
#     y_predA = ppn.predict(X_test)
    
#     cnf_matrix2 = confusion_matrix(y_test, y_predA)
#     ResPercep = 1 - (y_test != y_predA).sum()/y_test.shape[0]
#     ParcialesPercep.append(ResPercep)
#     print("Aplicando Perceptrón sobre datos luego del PCA")
#     print("Número de iteraciones")
#     print(ppn.n_iter_)
#     print("Cantidad de errores de clasificación sobre un total de {0} casos: {1}"
#             .format(y_test.shape[0],(y_test != y_predA).sum()))
#     print("Efectividad del algoritmo: {0: .2f}"
#             .format(ResPercep))
#     # Graficando la matriz de confusión
#     sns.heatmap(cnf_matrix2.T, square=True, annot=True, fmt='d', cbar=False)
#     plt.xlabel('Clase verdadera')
#     plt.ylabel('Clase predecida')
#     plt.title('Matriz de Confusión')
#     plt.show()
    
    
#     #Arbol de decisión
#     clf = DecisionTreeClassifier()
#     clf = clf.fit(X_train ,y_train)
#     y_predP = clf.predict(X_test)

#     cnf_matrix3 = confusion_matrix(y_test, y_predP)
#     ResArbol = 1 - (y_test != y_predP).sum()/y_test.shape[0]
#     ParcialesArbol.append(ResArbol)
#     print("Aplicando Árbol de decisión sobre datos luego del PCA")
#     print("Cantidad de errores de clasificación sobre un total de {0} casos: {1}"
#             .format(y_test.shape[0],(y_test != y_predP).sum()))
#     print("Efectividad del algoritmo: {0: .2f}"
#             .format(ResArbol))
#     # Graficando la matriz de confusión
#     sns.heatmap(cnf_matrix3.T, square=True, annot=True, fmt='d', cbar=False)
#     plt.xlabel('Clase verdadera')
#     plt.ylabel('Clase predecida')
#     plt.title('Matriz de Confusión')
#     plt.show()
    
#     #Ensamble sobre resultados parciales
    
#     for i in range(len(y_pred)):
#         y1 = y_pred[i]
#         y2 = y_predP[i]
#         y3 = y_predA[i]
    
# #     if y1 != y2 and y1 != y3 and y2 != y3:
# #         p1 = 1 - (y_test != y_pred).sum()/y_test.shape[0]
# #         p2 = 1 - (y_test != y_predA).sum()/y_test.shape[0]
# #         p3 = 1 - (y_test != y_predP).sum()/y_test.shape[0]
        
# #     else:
#         x = [y1, y2, y3]
#         y_pred_Ensamble.append(mode(x))

#     cnf_matrixEnsamble = confusion_matrix(y_test, y_pred_Ensamble)
#     ResEnsamble = 1 - (y_test != y_pred_Ensamble).sum()/y_test.shape[0]
#     ParcialesEnsamble.append(ResEnsamble)
#     print("Aplicando el ensamble, resultados parciales")
#     print("Cantidad de errores de clasificación sobre un total de {0} casos: {1}"
#             .format(y_test.shape[0],(y_test != y_pred_Ensamble).sum()))
#     print("Efectividad del algoritmo: {0: .2f}"
#             .format(ResEnsamble))
#     # Graficando la matriz de confusión
#     sns.heatmap(cnf_matrixEnsamble.T, square=True, annot=True, fmt='d', cbar=False)
#     plt.xlabel('Clase verdadera')
#     plt.ylabel('Clase predecida')
#     plt.title('Matriz de Confusión')
#     plt.show()
#     y_pred_Ensamble = []

# print("Resultados de efectividad promedio:")
# avgBay = sum(ParcialesBayes)/len(ParcialesBayes)
# avgPer = sum(ParcialesPercep)/len(ParcialesPercep)
# avgArb = sum(ParcialesArbol)/len(ParcialesArbol)
# avgEns = sum(ParcialesEnsamble)/len(ParcialesEnsamble)

# print("Naive Bayes: ")
# print(avgBay)

# print("Perceptrón: ")
# print(avgPer)

# print("Árbol de decisión: ")
# print(avgArb)

# print("Ensamble aplicado: ")
# print(avgEns)

# #     #Generar un árbol de decisión
# # from sklearn.tree import export_graphviz
# # from sklearn.externals.six import StringIO  
# # from IPython.display import Image  
# # import pydotplus

# # dot_data = StringIO()
# # export_graphviz(clf, out_file=dot_data,  
# #                 filled=True, rounded=True,
# #                 special_characters=True, class_names=['0','1','2','3','4','5','6'])
# # graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
# # graph.write_png("ArbolWPCA.png")
# # Image(graph.create_png())



# # plt.scatter(y_test, y_pred_Ensamble, jitter = True)
# # plt.show()
